# Laboratorio #1: Indexación

En esta clase práctica aprenderemos cómo funcionan los sistemas de búsqueda de información mediante la implementación de un índice invertido. Un índice invertido es una estructura de datos fundamental que permite realizar búsquedas eficientes en grandes colecciones de documentos, similar a cómo funciona un motor de búsqueda web.

Durante la práctica, construiremos paso a paso:
- Una estructura para representar documentos
- Un pipeline de preprocesamiento de texto
- Un índice invertido básico con posting lists
- Funcionalidad de búsqueda

Este sistema nos permitirá entender los conceptos fundamentales detrás de herramientas como Google, Elasticsearch, o cualquier motor de búsqueda moderno.

---

## Importación de Librerías

In [ ]:
%pip install -U spacy numpy matplotlib
%pip install ir-datasets

import spacy
from spacy.cli import download
download("en_core_web_sm")


In [57]:
import re
import json
import pickle
import time
from collections import defaultdict, Counter
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
import unicodedata
import matplotlib.pyplot as plt
import numpy as np
import math
import spacy

# Configuración de visualizaciones
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Obtención del dataset a utilizar

In [ ]:
import ir_datasets

cranfield_dataset = ir_datasets.load("cranfield")

## Estructura de Documentos

Definimos la estructura básica de un documento en nuestro sistema.

In [ ]:
@dataclass
class Document:
    """
    Estructura básica de un documento en el sistema DocuSearch
    
    Args:
        id: str. Identificador único del documento
        title: str. Título del documento (opcional)
        content: str. Contenido principal del documento
        author: str. Autor del documento (opcional)
        date: str. Fecha de publicación (opcional)
        category: str. Categoría del documento (opcional)
        metadata: Dict. Metadatos adicionales (opcional)
    """
    id: str
    title: str = ""
    content: str
    author: str = ""
    date: str = ""
    category: str = ""
    metadata: Dict = None
    
    def __post_init__(self):
        if self.metadata is None:
            self.metadata = {}
    
    def get_full_text(self) -> str:
        """
        Retorna el texto completo del documento (título + contenido)
        
        Returns:
            str. Texto completo combinando título y contenido
        """
        return f"{self.title} {self.content}"
    
    def get_word_count(self) -> int:
        """
        Retorna el número de palabras en el documento
        
        Returns:
            int. Número total de palabras
        """
        return len(self.get_full_text().split())
    
    def __str__(self):
        return f"Doc[{self.id}]: {self.title[:50]}{'...' if len(self.title) > 50 else ''}"

## Pipeline de Preprocesamiento

### ¿Qué es la tokenización y por qué la necesitamos?

La **tokenización** es el proceso de dividir un texto en unidades más pequeñas llamadas "tokens" (generalmente palabras). Es un paso fundamental en el procesamiento de lenguaje natural porque:

1. **Normalización**: Convierte el texto a una forma estándar (minúsculas, elimina puntuación)
2. **Limpieza**: Remueve elementos innecesarios como stopwords ("el", "la", "de", etc.)
3. **Reducción**: Mediante lematización, reduce palabras a su forma base (ej: "corriendo" → "correr")
4. **Eficiencia**: Permite crear índices más compactos y búsquedas más rápidas

Sin tokenización, no podríamos comparar eficientemente el texto de búsqueda con los documentos indexados.

Implementamos las etapas de procesamiento de texto usando SpaCy: tokenización, lematización y filtrado básico.

In [ ]:
class SpacyDocumentProcessor:
    """
    Procesador de documentos usando SpaCy para tokenización y normalización
    
    Args:
        remove_stopwords: bool. Si True, elimina stopwords del texto
        min_word_length: int. Longitud mínima de palabras a conservar
        use_lemma: bool. Si True, usa lematización
    """
    def __init__(self, 
                 remove_stopwords=True,
                 min_word_length=2,
                 use_lemma=True):
        
        self.nlp = spacy.load("en_core_web_sm")
        self.remove_stopwords = remove_stopwords
        self.min_word_length = min_word_length
        self.use_lemma = use_lemma

    def process_text(self, text: str):
        """
        Procesa un texto y retorna lista de términos normalizados
        
        Args:
            text: str. Texto a procesar
            
        Returns:
            list. Lista de términos procesados
        """
        doc = self.nlp(text)
        terms = []

        for token in doc:
            if token.is_punct or token.is_space:
                continue
            
            if self.remove_stopwords and token.is_stop:
                continue
            
            term = token.lemma_ if self.use_lemma else token.text
            term = term.lower()

            if len(term) >= self.min_word_length:
                terms.append(term)

        return terms

## Índice Invertido Simple con Posting Lists

### EJERCICIOS DE LA CLASE

En esta sección implementarás las funciones principales del índice invertido:

**EJERCICIO 1**: Completar la función `add_document()`

**EJERCICIO 2**: Completar la función `build_index()`

Busca los comentarios `# COMPLETAR AQUÍ` para saber dónde escribir el código.

---

Implementamos la estructura básica del índice invertido con posting lists.

In [ ]:
class SimpleInvertedIndex:
    """
    Implementación básica de un índice invertido con posting lists
    
    La estructura es: {término: {doc_id: frecuencia}}
    """
    
    def __init__(self):
        """
        Inicializa el índice invertido
        """
        self.index = defaultdict(dict)
        self.documents = {}           # doc_id -> Document object
        self.processor = SpacyDocumentProcessor()
        self.vocab_size = 0
        self.total_documents = 0
        
    
    # ============================================================================
    # EJERCICIO 1: COMPLETAR LA FUNCIÓN add_document()
    # ============================================================================
    def add_document(self, document: Document) -> None:
        """
        Añade un documento al índice creando las posting lists
        
        Args:
            document: Document. Documento a indexar
        """
        # ---------------------- COMPLETAR AQUÍ ----------------------
        # Paso 1: Procesar el texto del documento usando self.processor.process_text()
        # Asignar el resultado a la variable 'terms'
        terms = None  # REEMPLAZAR: obtener términos del documento
        
        # Paso 2: Contar las frecuencias de cada término
        # Usar Counter() para contar cuántas veces aparece cada término
        term_counts = None  # REEMPLAZAR: contar frecuencias
        
        # Paso 3: Añadir términos al índice
        # Para cada término y su frecuencia, agregar al índice invertido
        # self.index[term][document.id] = frecuencia
        for term, freq in ...:  # REEMPLAZAR: iterar sobre term_counts
            pass  # REEMPLAZAR: agregar al índice
        # ------------------------------------------------------------
        
        # Almacenar documento
        self.documents[document.id] = document
        
        # Actualizar estadísticas
        self.vocab_size = len(self.index)
        self.total_documents = len(self.documents)
        
    
    # ============================================================================
    # EJERCICIO 2: COMPLETAR LA FUNCIÓN build_index()
    # ============================================================================
    def build_index(self, documents: List[Document]) -> None:
        """
        Construye el índice desde una colección de documentos
        
        Args:
            documents: List[Document]. Lista de documentos a indexar
        """
        print(f"Iniciando indexación de {len(documents)} documentos...")
        start_time = time.time()
        
        for i, doc in enumerate(documents, 1):
            # ---------------------- COMPLETAR AQUÍ ----------------------
            # Llamar a add_document() para cada documento
            pass  # REEMPLAZAR: agregar documento al índice
            # ------------------------------------------------------------
            
            if i % 500 == 0:  # Progreso cada 500 documentos
                print(f"   Progreso: {i}/{len(documents)} documentos procesados")
        
        end_time = time.time()
        print(f"\nIndexación completada en {end_time - start_time:.2f} segundos")
        print(f"   Documentos: {self.total_documents}")
        print(f"   Vocabulario: {self.vocab_size} términos únicos")
    
    def search(self, query: str) -> List[Tuple[str, Document]]:
        """
        Realiza búsqueda simple en el índice usando posting lists
        
        Args:
            query: str. Consulta de búsqueda
            
        Returns:
            List[Tuple[str, Document]]. Lista de tuplas (doc_id, document) con resultados
        """
        # Procesar la consulta
        query_terms = self.processor.process_text(query)
        
        if not query_terms:
            print("La consulta no contiene términos válidos después del procesamiento")
            return []
        
        print(f"Buscando términos: {query_terms}")
        
        # Encontrar documentos que contienen TODOS los términos (AND)
        result_docs = None
        
        for term in query_terms:
            if term in self.index:
                term_docs = set(self.index[term].keys())
                if result_docs is None:
                    result_docs = term_docs
                else:
                    result_docs &= term_docs
            else:
                print(f"   Término '{term}' no encontrado en el índice")
                return []  # Si algún término no existe, no hay resultados
        
        if result_docs is None:
            return []
        
        # Convertir IDs a objetos Document
        results = [(doc_id, self.documents[doc_id]) for doc_id in result_docs]
        
        print(f"Encontrados {len(results)} documentos")
        return results
    
    def get_term_frequency(self, term: str) -> int:
        """
        Retorna el número de documentos que contienen un término
        
        Args:
            term: str. Término a buscar
            
        Returns:
            int. Número de documentos que contienen el término
        """
        return len(self.index.get(term, set()))
    
    def get_document(self, doc_id: str) -> Optional[Document]:
        """
        Recupera un documento por su ID
        
        Args:
            doc_id: str. ID del documento
            
        Returns:
            Document. Objeto Document o None si no existe
        """
        return self.documents.get(doc_id)
    
    def get_vocabulary(self) -> List[str]:
        """
        Retorna lista de todos los términos en el vocabulario
        
        Returns:
            List[str]. Lista de términos únicos
        """
        return list(self.index.keys())
    
    def get_statistics(self) -> Dict:
        """
        Retorna estadísticas básicas del índice
        
        Returns:
            Dict. Diccionario con estadísticas del índice
        """
        # Calcular distribución de frecuencias
        doc_frequencies = [len(doc_set) for doc_set in self.index.values()]
        
        return {
            'total_documents': self.total_documents,
            'vocabulary_size': self.vocab_size,
            'avg_terms_per_doc': np.mean([len(self.processor.process_text(doc.get_full_text())) 
                                         for doc in self.documents.values()]),
            'max_doc_frequency': max(doc_frequencies) if doc_frequencies else 0,
            'min_doc_frequency': min(doc_frequencies) if doc_frequencies else 0
        }
    

## Colección de Documentos

Utilizamos el dataset Cranfield para probar nuestro sistema.

In [ ]:
def create_documents(dataset) -> List[Document]:
    """
    Crea una colección de documentos desde el dataset Cranfield
    
    Args:
        dataset: Dataset. Dataset de ir_datasets
    
    Returns:
        List[Document]. Lista de documentos de prueba
    """
    
    documents = []
    for doc in dataset.docs_iter():
        # Implementar
        pass
    
    return documents

# Crear colección de documentos
sample_docs = create_documents(cranfield_dataset)
print(f"Colección creada con {len(sample_docs)} documentos")
print("\nPrimeros 3 documentos:")
for i, doc in enumerate(sample_docs[:3]):
    print(f"{i+1}. {doc}")

📚 Colección creada con 1400 documentos

Primeros 3 documentos:
1. Doc[1]: Doc. Title
2. Doc[2]: Doc. Title
3. Doc[3]: Doc. Title


## Construcción del Índice

Creamos e indexamos nuestra colección de documentos.

In [104]:
# Crear instancia del índice
docusearch = SimpleInvertedIndex()

# Construir índice
docusearch.build_index(sample_docs)

🚀 Iniciando indexación de 1400 documentos...
   Progreso: 500/1400 documentos procesados
   Progreso: 1000/1400 documentos procesados

🎉 Indexación completada en 24.16 segundos
   Documentos: 1400
   Vocabulario: 10319 términos únicos


## Pruebas de Búsqueda

Realizamos búsquedas de prueba para validar el funcionamiento del sistema.

In [ ]:
def show_search_results(results: List[Tuple[str, Document]], max_content_length: int = 100):
    """
    Muestra los resultados de búsqueda de forma legible
    
    Args:
        results: List[Tuple[str, Document]]. Lista de resultados de búsqueda
        max_content_length: int. Longitud máxima del contenido a mostrar
    """
    if not results:
        print("No se encontraron resultados")
        return
    
    print(f"\nRESULTADOS DE BÚSQUEDA ({len(results)} documentos)")
    print("=" * 60)
    
    for i, (doc_id, doc) in enumerate(results, 1):
        content_preview = doc.content[:max_content_length]
        if len(doc.content) > max_content_length:
            content_preview += "..."
        
        print(f"{i}. [{doc_id}] {doc.title}")
        print(f"   Autor: {doc.author} | Categoría: {doc.category} | Fecha: {doc.date}")
        print(f"   Contenido: {content_preview}")
        print()

In [ ]:
# Prueba 1: Búsqueda de "University"
print("BÚSQUEDA 1: 'University'")
results1 = docusearch.search("university")
show_search_results(results1)

🔍 BÚSQUEDA 1: 'inteligencia artificial'
🔍 Buscando términos: ['university']
📊 Encontrados 7 documentos

✅ RESULTADOS DE BÚSQUEDA (7 documentos)
1. [220] Doc. Title
   Autor:  | Categoría: cranfield | Fecha: 
   Contenido: a general purpose analogue correlator for the analysis of
random noise signals .
a large proportion ...

2. [610] Doc. Title
   Autor:  | Categoría: cranfield | Fecha: 
   Contenido: corner interference effects .
  the three-dimensional incompressible flow of fluid along the corner ...

3. [549] Doc. Title
   Autor:  | Categoría: cranfield | Fecha: 
   Contenido: experimental study of the velocity and temperature
distribution in a high-velocity vortex-type flow ...

4. [344] Doc. Title
   Autor:  | Categoría: cranfield | Fecha: 
   Contenido: some experimental techniques in mass transfer cooling .
  author introduces his survey by a brief re...

5. [1229] Doc. Title
   Autor:  | Categoría: cranfield | Fecha: 
   Contenido: the effect of sweep angle on hypersonic flow 

In [ ]:
# Prueba 2: Búsqueda de "Insects"
print("BÚSQUEDA 2: 'Insects'")
results2 = docusearch.search("insects")
show_search_results(results2)

🔍 BÚSQUEDA 2: 'tecnología'
🔍 Buscando términos: ['insects']
📊 Encontrados 1 documentos

✅ RESULTADOS DE BÚSQUEDA (1 documentos)
1. [933] Doc. Title
   Autor:  | Categoría: cranfield | Fecha: 
   Contenido: the characteristics of roughness from insects as observed
for two-dimensional, incompressible flow p...



In [ ]:
# Prueba 3: Búsqueda de "Health"
print("BÚSQUEDA 3: 'Health'")
results3 = docusearch.search("health")
show_search_results(results3)

🔍 BÚSQUEDA 3: 'medicina salud'
🔍 Buscando términos: ['health']
   Término 'health' no encontrado en el índice
❌ No se encontraron resultados


In [ ]:
# Prueba 4: Búsqueda sin resultados
print("BÚSQUEDA 4: 'Sports'")
results4 = docusearch.search("Sports")
show_search_results(results4)

🔍 BÚSQUEDA 4: 'término inexistente'
🔍 Buscando términos: ['sports']
   Término 'sports' no encontrado en el índice
❌ No se encontraron resultados


## EJERCICIO ADICIONAL: Implementación de Skip Lists

### EJERCICIO AVANZADO PARA ESTUDIO INDEPENDIENTE

Como ejercicio adicional y para profundizar en estructuras de datos avanzadas para sistemas de búsqueda, se propone implementar **Skip Lists** para optimizar las operaciones de intersección en las posting lists.

#### ¿Qué es una Skip List?

Una Skip List es una estructura de datos probabilística que permite realizar operaciones de búsqueda, inserción y eliminación en tiempo O(log n) en promedio. En el contexto de índices invertidos, las skip lists pueden acelerar significativamente las operaciones de intersección entre posting lists largas.

---

## EJERCICIOS PRÁCTICOS ADICIONALES

Los siguientes ejercicios te ayudarán a profundizar en conceptos clave de indexación y recuperación de información. Aplica todo lo que has aprendido sobre preprocesamiento de texto, posting lists y optimizaciones.

---

### EJERCICIO PRÁCTICO 1: Análisis de Stemming

**Situación:** Estás desarrollando un motor de búsqueda en español y has notado que el algoritmo de stemming que implementaste está agrupando algunas palabras que quizás no deberían estar juntas. 

Tu algoritmo reduce las siguientes parejas de palabras a la misma forma base:
- a. abandono/abandonamiento  
- b. absorción/absorbente
- c. mercadeo/mercados
- d. universidad/universo
- e. volumen/volúmenes

**Tu tarea:** 
1. **Analiza cada pareja** y determina cuáles NO deberían ser conflacionadas (agrupadas) por el stemmer.
2. **Justifica tu razonamiento** considerando:
   - ¿Tienen las palabras el mismo significado semántico?
   - ¿Un usuario que busque una palabra estaría interesado en resultados de la otra?
   - ¿Qué impacto tendría en la precisión vs recall del sistema?

**Reflexiona:** ¿Cómo podría este comportamiento del stemmer afectar la experiencia del usuario final en las búsquedas?

---

### EJERCICIO PRÁCTICO 2: Optimización de Intersección de Posting Lists

**Situación:** Tu motor de búsqueda debe procesar una consulta de dos términos que requiere intersección de posting lists. Has implementado tanto posting lists estándar como posting lists con skip pointers.

**Datos del problema:**
- **Término A** tiene una posting list con 16 entradas:  
  `[4, 6, 10, 12, 14, 16, 18, 20, 22, 32, 47, 81, 120, 122, 157, 180]`
  
- **Término B** tiene una posting list con 1 entrada:  
  `[47]`

**Tu tarea:**
1. **Calcula las comparaciones** necesarias para intersectar estas listas usando:
   - a) Posting lists estándar (sin skip pointers)
   - b) Posting lists con skip pointers (usando longitud de salto sugerida de √P, donde P es el tamaño de la lista)

2. **Justifica tus cálculos** paso a paso.

3. **Analiza el resultado:** 
   - ¿Cuándo son más efectivos los skip pointers? 
   - ¿Qué pasa cuando una lista es muy pequeña comparada con la otra?
   - ¿En qué escenarios los skip pointers no ofrecen ventajas?

**Bonus:** ¿Cómo cambiarían los cálculos si ambas listas tuvieran tamaños similares y grandes?